In [1]:
import pandas as pd

df = pd.read_csv('preprocessed_whole_dataset.csv')
df

,Unnamed: 0,선종,상태,state,vessel_type,nation_type
0,0,낚시어선,이동,1,8,1
1,1,낚시어선,조업,0,8,1
2,2,낚시어선,표류,2,8,1
3,3,등광조망,이동,1,2,0
4,4,등광조망,표류,2,2,0
...,...,...,...,...,...,...
1576,1576,범장망,이동,1,3,0
1577,1577,범장망,표류,2,3,0
1578,1578,유망,표류,2,1,0
1579,1579,타망,조업,0,0,0


In [2]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical


# 이미지 디렉토리 경로
image_dir = 'imgdata'  # 경로 수정 필요

# 이미지 읽기 및 전처리
images = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

train_data = images
train_data
target = df['vessel_type']
target = to_categorical(target, num_classes=10)
target

2024-10-21 07:33:22.516755: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-21 07:33:22.531913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 07:33:22.550008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 07:33:22.555439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 07:33:22.570360: I tensorflow/core/platform/cpu_feature_guar

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def ResNet50(input_shape=(256, 256, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    
    # MobileNetV2 backbone
    backbone = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(128, activation='softmax')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='ResNet50')
    return model

model = ResNet50()
model.summary()

2024-10-21 07:33:41.059477: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38716 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe MIG 3g.40gb, pci bus id: 0000:65:00.0, compute capability: 8.0


Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 23,798,154 (90.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
history = model.fit(train_data, target, epochs=280, verbose=1)

Epoch 1/280


I0000 00:00:1729496048.612067   16810 service.cc:146] XLA service 0x7fd8c00032f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729496048.612091   16810 service.cc:154]   StreamExecutor device (0): NVIDIA A100 80GB PCIe MIG 3g.40gb, Compute Capability 8.0
2024-10-21 07:34:09.409168: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-21 07:34:12.560293: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-10-21 07:34:16.626349: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16898', 4 bytes spill stores, 4 bytes spill loads

2024-10-21 07:34:17.479916: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory 

 1/50 ━━━━━━━━━━━━━━━━━━━━ 54:37 67s/step - accuracy: 0.0625 - loss: 2.3038

I0000 00:00:1729496091.676051   16810 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.2497 - loss: 2.2505

2024-10-21 07:35:02.359251: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16898', 4 bytes spill stores, 4 bytes spill loads



50/50 ━━━━━━━━━━━━━━━━━━━━ 109s 857ms/step - accuracy: 0.2503 - loss: 2.2499
Epoch 2/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2567 - loss: 2.1913
Epoch 3/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2830 - loss: 2.1506
Epoch 4/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2740 - loss: 2.1295
Epoch 5/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2664 - loss: 2.1072
Epoch 6/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2701 - loss: 2.0967
Epoch 7/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2831 - loss: 2.0622
Epoch 8/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2504 - loss: 2.0717
Epoch 9/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2829 - loss: 2.0736
Epoch 10/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2889 - loss: 2.0313
Epoch 11/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.2832 - loss: 2.0245
Epoch 12/280
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accura

In [6]:
# import os
# import numpy as np
# from PIL import Image
# from tensorflow.keras.preprocessing.image import img_to_array

# test_image_dir = '테스트셋'  # 이미지 폴더 경로

# # 이미지 읽기 및 전처리
# test_images = []  # 리스트로 선언
# for filename in os.listdir(test_image_dir):
#     if filename.lower().endswith(('.jpg', '.png')):  # 확장자를 소문자로 변환 후 비교
#         img_path = os.path.join(test_image_dir, filename)
#         try:
#             image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
#             image_array = img_to_array(image) / 255.0  # 이미지를 배열로 변환하고 정규화
#             test_images.append(image_array)  # 리스트에 이미지 추가
#         except Exception as e:
#             print(f"Error loading image {filename}: {e}")

# # 이미지 데이터를 NumPy 배열로 변환
# test_images = np.array(test_images)  # 리스트를 한 번에 NumPy 배열로 변환

# test_data = test_images
# test_data

In [7]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

test_image_dir = '테스트셋'  # 이미지 폴더 경로

# 이미지 읽기 및 전처리
test_images = []  # 리스트로 선언

# 파일 목록을 가져오고 숫자 순서대로 정렬
file_list = sorted(os.listdir(test_image_dir), key=lambda x: int(os.path.splitext(x)[0]))

for filename in file_list:
    if filename.lower().endswith(('.jpg', '.png')):  # 확장자를 소문자로 변환 후 비교
        img_path = os.path.join(test_image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 이미지를 배열로 변환하고 정규화
            test_images.append(image_array)  # 리스트에 이미지 추가
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
test_images = np.array(test_images)  # 리스트를 한 번에 NumPy 배열로 변환

test_data = test_images
print(test_data)


[[[[0.7764706  0.75686276 0.73333335]
   [0.3882353  0.36862746 0.3529412 ]
   [0.26666668 0.24705882 0.24705882]
   ...
   [0.26666668 0.25882354 0.26666668]
   [0.26666668 0.25882354 0.27058825]
   [0.27058825 0.2627451  0.2784314 ]]

  [[0.7764706  0.7607843  0.7372549 ]
   [0.21568628 0.20392157 0.19215687]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.00392157]]

  [[0.7764706  0.7647059  0.7372549 ]
   [0.23137255 0.21960784 0.2       ]
   [0.01960784 0.00392157 0.00392157]
   ...
   [0.00784314 0.         0.00784314]
   [0.01176471 0.00392157 0.01568628]
   [0.01568628 0.00784314 0.02352941]]

  ...

  [[0.76862746 0.77254903 0.7411765 ]
   [0.21568628 0.21568628 0.20392157]
   [0.         0.         0.        ]
   ...
   [0.00784314 0.         0.01568628]
   [0.00784314 0.         0.02352941]
   [0.01176471 0.         0.02745098]]

  [[0.76862746 0.77254903 0.7411765 ]
   [0.2

In [8]:
predictions = model.predict(test_data)
# binary_predictions = (predictions > 0.5).astype(int)
class_predictions = np.argmax(predictions, axis=1)

print(class_predictions)
len(class_predictions)

1/2 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

2024-10-21 07:55:32.903749: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1679', 196 bytes spill stores, 196 bytes spill loads

2024-10-21 07:55:33.026375: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1679', 12 bytes spill stores, 12 bytes spill loads



2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 7s/step
[6 8 6 8 0 2 1 0 7 9 2 2 8 0 1 1 0 1 8 3 9 1 1 6 5 3 8 0 2 3 9 1 3 0 3 3 7
 6 1 6 7 8 8 0 0 3 1 0 3 3 0 0 3]


53